In [ ]:
import os
import numpy as np
import glob
import sys
import math
import json

from utils import *
from unet_model import *

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Model Loading 

In [ ]:
with open(glob.glob("info*")[0], "r") as f:
    info = json.load(f)
print(info)

model = UNet(input_dim=info["input_dim"], n_filters=info["n_filters"], kernel_size=info["kernel_size"],
                     n_depth = info["n_depth"], activation=info["activation"], norm=info["norm"], dropout_rate=info["dropout_rate"],
                     down_type=info["downtype"], up_type=info["uptype"], residual=info["residual"], SE=info["SE"])
model.load_weights("A_best.h5")

## Data Loading & Prediction

In [ ]:
protein = "4jwk"
dxname="gist-dA"
path_dir_data = "/hogehoge/"
path_pdb = path_dir_data + f"{protein}.pdb"

# Voxelization of protein 3D structures
a = Atoms()
a.voxelize(protein, path_pdb)
print("Load", a.box.shape, end='>> ')

a.split()
print("Split", a.small_box.shape, end='>> ')

# Prediction
gpred = model.predict(a.small_box)
print("Predict", gpred.shape, end='>> ')

gp = GistMap()
gp.recreate_from_small_box(small_box=gpred, ref=a)

print("Recreate ", gp.box_recreate.shape)

#Output
gp.to_dx(save_dir=f"./{protein}-{dxname}-pred.dx")